In [1]:
import numpy as np 
import pandas as pd
import xgboost as xgb
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix 
import pickle

In [4]:
X = pd.read_pickle('/kaggle/input/amex-imputed-aggregate-data/X_train_agg.pkl', compression='gzip') 
y = pd.read_pickle('/kaggle/input/amex-imputed-aggregate-data/y_train_agg.pkl', compression='gzip')             

In [8]:
display(X.head(10))
display(y.head(10))

,B_10_last,B_10_max,B_10_mean,B_10_min,B_10_std,B_11_last,B_11_max,B_11_mean,B_11_min,B_11_std,...,S_8_last,S_8_max,S_8_mean,S_8_min,S_8_std,S_9_last,S_9_max,S_9_mean,S_9_min,S_9_std
customer_ID,,,,,,,,,,,,,,,,,,,,,
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.326172,0.741699,0.270264,0.096191,0.181835,0.010262,0.010262,0.007229,0.002748,0.003031,...,0.488281,1.001953,0.796875,0.488281,0.136276,0.007397,0.093933,0.039825,0.007397,0.026709
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5,0.297119,0.302734,0.298828,0.293945,0.003044,0.014572,0.081238,0.013786,0.000416,0.021039,...,0.406494,0.763672,0.411133,0.002707,0.243573,0.127808,0.127808,0.033813,0.006783,0.052707
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1,0.296387,0.302734,0.273682,0.162109,0.052867,0.005093,0.007618,0.004684,0.000111,0.002311,...,0.009186,0.009521,0.006432,0.000590,0.002996,0.051758,0.153198,0.074890,0.026932,0.055494
000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc,0.411621,0.431885,0.306641,0.192993,0.079525,0.005489,0.241333,0.044281,0.000671,0.071059,...,0.170532,0.481201,0.308594,0.167847,0.129054,0.011429,0.031250,0.016891,0.005058,0.008305
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad51ca8b8c4a24cefed,0.125244,0.260742,0.100342,0.044739,0.074579,0.001000,0.009804,0.005016,0.000714,0.003694,...,0.008591,0.330078,0.053467,0.000093,0.122191,0.051758,0.153198,0.074890,0.026932,0.055494
000084e5023181993c2e1b665ac88dbb1ce9ef621ec5370150fc2f8bdca6202c,0.292969,0.302002,0.289062,0.241577,0.020943,0.004299,0.030823,0.011551,0.002266,0.009107,...,0.004009,0.472656,0.249268,0.000986,0.161978,0.051758,0.153198,0.074890,0.026932,0.055494
000098081fde4fd64bc4d503a5d6f86a0aedc425c96f5235f98b0f47c9d7d8d4,0.267090,0.267090,0.073181,0.020004,0.069608,0.003668,0.075867,0.045105,0.003668,0.023830,...,0.007240,0.408691,0.214600,0.004200,0.143547,0.013107,0.013107,0.013107,0.013107,0.055494
0000d17a1447b25a01e42e1ac56b091bb7cbb06317be4cb59b50fec59e0b6381,0.114136,0.292480,0.245972,0.090576,0.065897,0.002827,0.067993,0.039154,0.002827,0.025504,...,0.326904,0.603516,0.364258,0.119873,0.127855,0.051758,0.153198,0.074890,0.026932,0.055494
0000f99513770170a1aba690daeeb8a96da4a39f11fc27da5c30a79db61c1e85,0.010414,0.020737,0.011734,0.004711,0.005386,0.996582,0.996582,0.948730,0.919434,0.026161,...,0.594727,0.602539,0.163696,0.002962,0.250804,0.015900,0.015900,0.015900,0.015900,0.055494


customer_ID
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a    0
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5    0
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1    0
000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc    0
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad51ca8b8c4a24cefed    0
000084e5023181993c2e1b665ac88dbb1ce9ef621ec5370150fc2f8bdca6202c    0
000098081fde4fd64bc4d503a5d6f86a0aedc425c96f5235f98b0f47c9d7d8d4    0
0000d17a1447b25a01e42e1ac56b091bb7cbb06317be4cb59b50fec59e0b6381    0
0000f99513770170a1aba690daeeb8a96da4a39f11fc27da5c30a79db61c1e85    1
00013181a0c5fc8f1ea38cd2b90fe8ad2fa8cad9d9f13e4063bdf6b0f7d51eb6    1
Name: target, dtype: int8

In [9]:
#kaggle competition metric

def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:

    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
        
    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)

    return 0.5 * (g + d)

In [10]:
xgb_classifier = xgb.XGBClassifier() 

In [11]:
xgb_classifier = xgb_classifier.fit(X,y)

In [13]:
proba_preds = xgb_classifier.predict_proba(X)[:, 1]
preds = xgb_classifier.predict(X)

In [14]:
acc = accuracy_score(y, preds)
print(acc)

0.9227457056130465


In [16]:
print(f'M = {amex_metric(pd.DataFrame(y), pd.DataFrame(proba_preds, index=y.index, columns=["prediction"]))}')

M = 0.8517121892133738


In [33]:
preds = pd.DataFrame(preds) #convert numpy array to DataFrame

In [36]:
print(classification_report(y, preds))

              precision    recall  f1-score   support

           0       0.95      0.95      0.95    340085
           1       0.85      0.85      0.85    118828

    accuracy                           0.92    458913
   macro avg       0.90      0.90      0.90    458913
weighted avg       0.92      0.92      0.92    458913



In [44]:
X_test = pd.read_pickle('/kaggle/input/amex-imputed-aggregate-data/X_test_agg.pkl', compression='gzip')

In [46]:
submission = pd.DataFrame(xgb_classifier.predict_proba(X_test)[:, 1], index=X_test.index, columns=['prediction'])

# index needs to be removed from submission csv
submission = submission.reset_index()
submission.to_csv('xgb_classification_baseline_submission.csv', index=False)